In [2]:
# %%
# Step 1: Import libraries
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from dotenv import load_dotenv

# %%
# Step 2: Load environment variables
load_dotenv()

pg_user = os.environ['PG_USER']
pg_password = os.environ['PG_PASSWORD']
pg_host = os.environ['PG_HOST']
pg_port = os.environ['PG_PORT']
pg_db = os.environ['PG_DB']

# %%

In [3]:
# Step 3: Create engine
from sqlalchemy import create_engine

engine = create_engine(
    f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}'
)


## Diagnostic

### Business Question
What are the most common URL feature combinations (HTTPS usage and IP address presence) among verified phishing URLs?

In [4]:
query1 = '''
WITH length_buckets AS (
    SELECT 
        CASE 
            WHEN u.url_length < 30 THEN 'Short'
            WHEN u.url_length BETWEEN 30 AND 60 THEN 'Medium'
            ELSE 'Long'
        END AS length_category,
        f.phishing_status,
        COUNT(*) AS url_count
    FROM raw.fact_phishing_urls f
    JOIN raw.dim_url_features u
        ON f.feature_id = u.feature_id
    GROUP BY length_category, f.phishing_status
),
ranked_lengths AS (
    SELECT *,
           RANK() OVER (PARTITION BY phishing_status ORDER BY url_count DESC) AS rank_within_status
    FROM length_buckets
)
SELECT *
FROM ranked_lengths
WHERE phishing_status = 'Phishing (Verified)'
ORDER BY url_count DESC;

'''

df_q1 = pd.read_sql(query1, con=engine)
df_q1


,length_category,phishing_status,url_count,rank_within_status
0,Medium,Phishing (Verified),47,1
1,Short,Phishing (Verified),10,2


### Insight  
Among verified phishing URLs, the **"Medium" length category (30–60 characters)** is the most common, with 47 instances, compared to only 10 in the "Short" category. No long URLs appeared in this filtered result set, suggesting that phishers tend to use medium-length URLs when crafting deceptive links.

### Recommendation  
Awareness campaigns and security tools should pay closer attention to medium-length URLs that appear suspicious or unfamiliar. URL length, while not a definitive indicator of phishing, can be used as a risk signal when combined with other red flags like lack of HTTPS or suspicious domains.

### Prediction  
If the trend continues, phishing detection models that factor in **medium-length URLs** as a high-risk feature may improve early identification of phishing attacks, especially when integrated with additional metadata like domain age or SSL usage.


## Descriptive

### Business Question
What are the most common URL lengths among verified phishing websites?

In [5]:
query2 = '''
SELECT
    u.url_length,
    COUNT(*) AS url_count
FROM raw.fact_phishing_urls f
JOIN raw.dim_url_features u 
    ON f.feature_id = u.feature_id
WHERE f.phishing_status = 'Phishing (Verified)'
GROUP BY u.url_length
ORDER BY url_count DESC
LIMIT 7;
'''

df_q2 = pd.read_sql(query2, con=engine)
df_q2


,url_length,url_count
0,32,7
1,33,5
2,31,5
3,34,5
4,30,5
5,39,4
6,37,3


### Insight
Among phishing URLs, the most frequently observed lengths are between 30 and 34 characters, with 32-character URLs being the most common.

### Recommendation
When building detection rules or URL scanning tools, prioritize flagging URLs that fall within this 30–34 character range, especially if other suspicious patterns (like lack of HTTPS or use of IPs) are also present.

### Prediction
Phishing attempts will likely continue to use URL lengths that resemble typical login or short redirect links (30–35 characters) in order to appear legitimate to users and evade detection heuristics.
